# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parameterized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## Layout of the SageMaker ModelBuild Project Template

The template provides a starting point for bringing your SageMaker Pipeline development to production.

```
|-- codebuild-buildspec.yml
|-- CONTRIBUTING.md
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- README.md
|-- sagemaker-pipelines-project.ipynb
|-- setup.cfg
|-- setup.py
|-- tests
|   `-- test_pipelines.py
`-- tox.ini
```

A description of some of the artifacts is provided below:
<br/><br/>
Your codebuild execution instructions:
```
|-- codebuild-buildspec.yml
```
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the Mean Squared Error of the model that's trained by the pipeline:

```
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py

```

For additional subfolders with code and/or artifacts needed by pipeline, they need to be packaged correctly by the `setup.py` file. For example, to package a `pipelines/source` folder,

* Include a `__init__.py` file within the `source` folder.
* Add it to the `setup.py` file's `package_data` like so:

```
...
    packages=setuptools.find_packages(),
    include_package_data=True,
    package_data={"pipelines.my_pipeline.src": ["*.txt"]},
    python_requires=">=3.6",
    install_requires=required_packages,
    extras_require=extras,
...
```

<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```
<br/><br/>
Python package artifacts:
```
|-- setup.cfg
|-- setup.py
```
<br/><br/>
A stubbed testing module for testing your pipeline as you develop:
```
|-- tests
|   `-- test_pipelines.py
```
<br/><br/>
The `tox` testing framework configuration:
```
`-- tox.ini
```

### AutoReload

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!pip install sagemaker --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/pipeline-full.png)

### Getting some constants

We get some constants from the local execution environment.

In [7]:
import boto3
import sagemaker
from utils.ssm import parameter_store

region = boto3.Session().region_name
pm = parameter_store(region)


# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team
model_package_group_name = f"NeMoASRModelPackageGroup-Example"
#pipeline_name = f"NeMoASRPipeline-Example"
base_job_prefix = pm.get_params(key='PREFIX')
role = pm.get_params(key=base_job_prefix + 'SAGEMAKER-ROLE-ARN') #sagemaker.get_execution_role()
default_bucket = pm.get_params(key=base_job_prefix + 'BUCKET')  # sagemaker.session.Session().default_bucket()


### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [102]:
@conf 수정하는거 추가하기

SyntaxError: invalid syntax (57024074.py, line 1)

In [8]:
from pipelines.nemo_asr.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    input_data_path = pm.get_params(key=base_job_prefix + 'S3-DATA-PATH'),
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    #pipeline_name=pipeline_name,
    base_job_prefix=pm.get_params(key="PREFIX")
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


strFilePath /home/ec2-user/SageMaker/nemo-on-sagemaker/pipelines/nemo_asr/config
====== config info. ======
  COMMON: prefix:SM-MLOPS-PIPELINE-
  COMMON: model_name:model-230304
  PIPELINE: name:NeMoASRPipeline-Example
  PIPELINE: enable_caching:True
  PIPELINE: expire_after:T48H
  PREPROCESSING: instance_type:ml.g4dn.xlarge
  PREPROCESSING: instance_count:1
  TRAINING: framework_version:None
  TRAINING: py_version:None
  TRAINING: instance_type:ml.p3.2xlarge
  TRAINING: instance_count:1
  EVALUATION: instance_type:ml.g4dn.xlarge
  EVALUATION: instance_count:1
  MODEL_REGISTER: model_package_group_name:NeMoASRModelPackageGroup-Example
  MODEL_REGISTER: model_approval_status_default:PendingManualApproval
  MODEL_REGISTER: inference_instances:["ml.p3.2xlarge"]
  MODEL_REGISTER: transform_instances:["ml.p3.2xlarge"]
  DEPLOY: processing_instance_type:ml.m5.xlarge
  DEPLOY: processing_instance_count:1
  DEPLOY: processing_framework_version:1.0-1
  DEPLOY: instance_type:ml.g4dn.xlarge
  DEP

INFO:sagemaker.processing:Uploaded ./an4_nemo_sagemaker/code/preprocessing/ to s3://sm-nemo-bucket/preprocessing/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-bucket/preprocessing/source/runproc.sh
INFO:sagemaker.processing:Uploaded ./an4_nemo_sagemaker/code/preprocessing/ to s3://sm-nemo-bucket/preprocessing-2/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-bucket/preprocessing-2/source/runproc.sh
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


   
Args:  dict_items([('ProcessingResources', {'ClusterConfig': {'InstanceType': 'ml.g4dn.xlarge', 'InstanceCount': 1, 'VolumeSizeInGB': 30}}), ('AppSpecification', {'ImageUri': '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-image:latest', 'ContainerArguments': ['--proc_prefix', '/opt/ml/processing', '--region', 'ap-northeast-2', '--train_mount_dir', '/opt/ml/input/data/training/', '--test_mount_dir', '/opt/ml/input/data/testing/'], 'ContainerEntrypoint': ['/bin/bash', '/opt/ml/processing/input/entrypoint/runproc.sh']}), ('RoleArn', 'arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436'), ('ProcessingInputs', [{'InputName': 'input-data', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sm-nemo-bucket/data', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://s

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


...

   
Args:  dict_items([('AlgorithmSpecification', {'TrainingInputMode': 'File', 'TrainingImage': '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-image:latest', 'MetricDefinitions': [{'Name': 'train_loss', 'Regex': 'loss.*=\\D*(.*?)$'}]}), ('OutputDataConfig', {'S3OutputPath': Join(on='/', values=['s3://sm-nemo-bucket', 'NeMoASRPipeline-Example', 'training', 'model-output'])}), ('StoppingCondition', {'MaxRuntimeInSeconds': 3600}), ('ResourceConfig', {'VolumeSizeInGB': 1024, 'InstanceCount': 1, 'InstanceType': 'ml.p3.2xlarge'}), ('RoleArn', 'arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436'), ('InputDataConfig', [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': <sagemaker.workflow.properties.Properties object at 0x7f591598af70>, 'S3DataDistributionType': 'FullyReplicated'}}, 'ChannelName': 'training'}, {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': <sagemaker.workflow.properties.Properties object at 

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.processing:Uploaded ./an4_nemo_sagemaker/code/evaluation/ to s3://sm-nemo-bucket/SM-NeMo-nemo-asr-exp-0318-15051679151941/source/sourcedir.tar.gz


  
== Evaluation Step ==


INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-bucket/SM-NeMo-nemo-asr-exp-0318-15051679151941/source/runproc.sh


   
Args:  dict_items([('ProcessingResources', {'ClusterConfig': {'InstanceType': 'ml.g4dn.xlarge', 'InstanceCount': 1, 'VolumeSizeInGB': 30}}), ('AppSpecification', {'ImageUri': '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-image:latest', 'ContainerEntrypoint': ['/bin/bash', '/opt/ml/processing/input/entrypoint/runproc.sh']}), ('RoleArn', 'arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436'), ('ProcessingInputs', [{'InputName': 'model_artifact', 'AppManaged': False, 'S3Input': {'S3Uri': <sagemaker.workflow.properties.Properties object at 0x7f59179c8e20>, 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'test_manifest_file', 'AppManaged': False, 'S3Input': {'S3Uri': Join(on='/', values=[<sagemaker.workflow.properties.Properties object at 0x7f591598af70>, 'an4', 'test_manifest.json']), 'LocalPath': '/opt/ml/pr

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [9]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded ./an4_nemo_sagemaker/code/preprocessing/ to s3://sm-nemo-bucket/NeMoASRPipeline-Example/code/cbd5532ffc88b3d0e0276a0c2a093982/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-bucket/NeMoASRPipeline-Example/code/5b4247ccf769254fe9ac221423614408/runproc.sh
INFO:sagemaker.processing:Uploaded ./an4_nemo_sagemaker/code/preprocessing/ to s3://sm-nemo-bucket/NeMoASRPipeline-Example/code/cbd5532ffc88b3d0e0276a0c2a093982/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-bucket/NeMoASRPipeline-Example/code/5b4247ccf769254fe9ac221423614408/runproc.sh
INFO:sagemaker.processing:Uploaded ./an4_nemo_sagemaker/code/evaluation/ to s3://sm-nemo-bucket/NeMoASRPipeline-Example/code/bf7fe1bb90ac8aeb9235a9d6b1616602/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-bucket/NeMoASRPipeline-Example/code/042c27441c10cac7d270da211efc863b/runproc.sh
INFO:sagemaker.processing:Uploaded ./an4_ne

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/nemoasrpipeline-example',
 'ResponseMetadata': {'RequestId': '309860dc-ff9b-477b-a0a0-1903d973ee31',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '309860dc-ff9b-477b-a0a0-1903d973ee31',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '96',
   'date': 'Sat, 18 Mar 2023 15:05:54 GMT'},
  'RetryAttempts': 0}}

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [10]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [86]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/nemoasrpipeline-example',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/nemoasrpipeline-example/execution/o7rk7bvt1dsj',
 'PipelineExecutionDisplayName': 'execution-1679148623045',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'nemoasrpipeline-example',
  'TrialName': 'o7rk7bvt1dsj'},
 'CreationTime': datetime.datetime(2023, 3, 18, 14, 10, 22, 978000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 3, 18, 14, 10, 22, 978000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '5bcf78e4-51ba-4271-9575-3550203b2d0f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5bcf78e4-51ba-4271-9575-3550203b2d0f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '520',
   'date': 'Sat, 18 Mar 2023 14:10:23 GMT'},
  'RetryAttempts': 0}}

We can wait for the execution by invoking `wait()` on the execution:

In [87]:
execution.wait()

KeyboardInterrupt: 

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

### Parameterized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()